# Zadanie 1: Rozpoznanie arytmetyki

---

## Część 1: Wyznaczanie epsilona maszynowego (`macheps`)

### 1. Opis problemu
Wyznaczenie epsilona maszynowego (`macheps`), czyli najmniejszej liczby $x > 0$ takiej, że $1.0 + x \neq 1.0$. Użyjemy iteracyjnego dzielenia przez 2.0. Porównamy wynik z funkcją `eps(T)` oraz standardowymi stałymi z pliku `float.h` języka C.

In [12]:
using Printf

function find_macheps(T::Type{<:AbstractFloat})
    one::T = T(1.0)
    two::T = T(2.0)
    macheps::T = one

    while (one + (macheps / two)) > one
        macheps = macheps / two
    end
    
    return macheps
end

types_to_test = [Float16, Float32, Float64]
println("--- Wyznaczanie Epsilona Maszynowego (macheps) ---")

for T in types_to_test
    @printf "--- Typ: %s ---\n" T
    @printf "Iteracyjnie: \t\t%.8e\n" find_macheps(T)
    @printf "Wbudowane (eps): \t%.8e\n" eps(T)
end

--- Wyznaczanie Epsilona Maszynowego (macheps) ---
--- Typ: Float16 ---
Iteracyjnie: 		9.76562500e-04
Wbudowane (eps): 	9.76562500e-04
--- Typ: Float32 ---
Iteracyjnie: 		1.19209290e-07
Wbudowane (eps): 	1.19209290e-07
--- Typ: Float64 ---
Iteracyjnie: 		2.22044605e-16
Wbudowane (eps): 	2.22044605e-16


### 3. Wyniki i interpretacja
**Interpretacja i porównanie z `float.h` (C):**
Wyniki obliczeń iteracyjnych są identyczne z funkcjami wbudowanymi Julii. Poniżej jawne zestawienie ze standardowymi stałymi C (używając precyzji z naszego outputu):

* **Float32 (Single):**
    * Nasz wynik: `1.19209290e-07`
    * Stała C `FLT_EPSILON`: `1.19209290e-07` ($2^{-23}$)
    * *Zgodność: Tak*

* **Float64 (Double):**
    * Nasz wynik: `2.22044605e-16`
    * Stała C `DBL_EPSILON`: `2.22044605e-16` ($2^{-52}$)
    * *Zgodność: Tak*

### 4. Wnioski
Obliczenia iteracyjne dały wyniki w pełni zgodne z wbudowanymi funkcjami Julii oraz standardowymi wartościami `FLT_EPSILON` i `DBL_EPSILON` ze **standardu IEEE 754**. Wartość ta definiuje *jednostkowy błąd zaokrąglenia* ($u = \text{macheps} / 2$), który jest kluczowym parametrem w analizie błędów **obliczeń maszynowych** i pokazuje granicę precyzji względnej operacji.

---

## Część 2: Wyznaczanie najmniejszej liczby dodatniej (`eta` / $MIN_{sub}$)

### 1. Opis problemu
Wyznaczenie najmniejszej dodatniej liczby maszynowej `eta` (tzw. $MIN_{sub}$). Użyjemy iteracyjnego dzielenia przez 2.0, aż do osiągnięcia 0.0. Porównamy wynik z `nextfloat(T(0.0))` oraz sprawdzimy, czy `float.h` definiuje tę wartość.

In [13]:
function find_eta(T::Type{<:AbstractFloat})
    one::T = T(1.0)
    two::T = T(2.0)
    zero::T = T(0.0)
    eta::T = one

    while (eta / two) > zero
        eta = eta / two
    end
    
    return eta
end

println("\n--- Wyznaczanie Najmniejszej Liczby Dodatniej (eta / MIN_sub) ---")

for T in types_to_test
    @printf "--- Typ: %s ---\n" T
    @printf "Iteracyjnie: \t\t\t%.8e\n" find_eta(T)
    @printf "Wbudowane (nextfloat(0.0)): \t%.8e\n" nextfloat(T(0.0))
end


--- Wyznaczanie Najmniejszej Liczby Dodatniej (eta / MIN_sub) ---
--- Typ: Float16 ---
Iteracyjnie: 			5.96046448e-08
Wbudowane (nextfloat(0.0)): 	5.96046448e-08
--- Typ: Float32 ---
Iteracyjnie: 			1.40129846e-45
Wbudowane (nextfloat(0.0)): 	1.40129846e-45
--- Typ: Float64 ---
Iteracyjnie: 			4.94065646e-324
Wbudowane (nextfloat(0.0)): 	4.94065646e-324


### 3. Wyniki i interpretacja
**Interpretacja i porównanie z `float.h` (C):**
Wyniki iteracyjne są identyczne z wartościami `nextfloat(T(0.0))`.

* **Float32 (Single):**
    * Nasz wynik (`eta`): `1.40129846e-45`
    * Stała C `float.h`: *Brak* (standard C definiuje tylko `FLT_MIN`, patrz Część 3)

* **Float64 (Double):**
    * Nasz wynik (`eta`): `4.94065646e-324`
    * Stała C `float.h`: *Brak* (standard C definiuje tylko `DBL_MIN`, patrz Część 3)

### 4. Wnioski
Eksperyment potwierdził istnienie **liczb subnormalnych (zdenormalizowanych)**, kluczowego elementu **standardu IEEE 754**. Wyznaczona `eta` ($MIN_{sub}$) reprezentuje mechanizm "łagodnego niedomiaru" (gradual underflow). W **obliczeniach maszynowych** pozwala to uniknąć gwałtownego "przeskoku" do zera, co jest krytyczne dla stabilności algorytmów operujących na bardzo małych wartościach.

---

## Część 3: Analiza `floatmin` vs $MIN_{nor}$ (i `float.h`)

### 1. Opis problemu
Sprawdzenie, co zwraca funkcja `floatmin(T)` i jaki ma związek z $MIN_{nor}$ (najmniejszą liczbą znormalizowaną) oraz ze stałymi `FLT_MIN` i `DBL_MIN` z `float.h`.

In [16]:
println("--- Badanie floatmin(T) ---")
# Uwaga: funkcja find_eta() została zdefiniowana w poprzedniej komórce
@printf "floatmin(Float16): \t%.8e (vs eta: %.8e)\n" floatmin(Float16) find_eta(Float16)
@printf "floatmin(Float32): \t%.8e (vs eta: %.8e)\n" floatmin(Float32) find_eta(Float32)
@printf "floatmin(Float64): \t%.8e (vs eta: %.8e)\n" floatmin(Float64) find_eta(Float64)

--- Badanie floatmin(T) ---
floatmin(Float16): 	6.10351562e-05 (vs eta: 5.96046448e-08)
floatmin(Float32): 	1.17549435e-38 (vs eta: 1.40129846e-45)
floatmin(Float64): 	2.22507386e-308 (vs eta: 4.94065646e-324)


### 3. Wyniki i interpretacja
**Interpretacja i porównanie z `float.h` (C):**
Wartości `floatmin(T)` są znacznie większe niż `eta` ($MIN_{sub}$). Porównanie ze stałymi C:

* **Float32 (Single):**
    * Nasz wynik (`floatmin`): `1.17549435e-38`
    * Stała C `FLT_MIN`: `1.17549435e-38`
    * *Zgodność: Tak*

* **Float64 (Double):**
    * Nasz wynik (`floatmin`): `2.22507386e-308`
    * Stała C `DBL_MIN`: `2.22507386e-308`
    * *Zgodność: Tak*

### 4. Wnioski
Eksperyment jasno pokazuje fundamentalne rozróżnienie w **standardzie IEEE 754**:
1.  `eta` ($MIN_{sub}$): Najmniejsza liczba subnormalna (obszar *gradual underflow*).
2.  `floatmin(T)` ($MIN_{nor}$): Najmniejsza liczba **znormalizowana**.

Jak widać z jawnego porównania, to właśnie $MIN_{nor}$ (a nie $MIN_{sub}$) jest zdefiniowane w standardzie C (`FLT_MIN`, `DBL_MIN`). W **obliczeniach maszynowych** $MIN_{nor}$ wyznacza początek zakresu, w którym liczby mają pełną precyzję względną.

---

## Część 4: Wyznaczanie największej liczby skończonej (`MAX`)

### 1. Opis problemu
Wyznaczenie największej skończonej liczby maszynowej ($MAX$). Znajdziemy największą potęgę dwójki $P = 2^{E_{max}}$ (mnożąc do `Inf`), a następnie obliczymy $MAX = P \times (2.0 - \text{macheps})$. Porównamy wynik z `floatmax(T)` oraz stałymi z `float.h`.

In [17]:
function find_max(T::Type{<:AbstractFloat})
    one::T = T(1.0)
    two::T = T(2.0)

    # Używamy funkcji zdefiniowanej w Części 1
    macheps = find_macheps(T) 

    # Znajdź największą potęgę dwójki (2^E_max)
    max_power_of_2 = one
    while !isinf(max_power_of_2 * two)
        max_power_of_2 = max_power_of_2 * two
    end
    
    # Oblicz MAX
    iterative_max = max_power_of_2 * (two - macheps)
    
    return iterative_max
end

println("\n--- Wyznaczanie Największej Liczby Skończonej (MAX) ---")

for T in types_to_test
    @printf "--- Typ: %s ---\n" T
    @printf "Iteracyjnie: \t\t%.8e\n" find_max(T)
    @printf "Wbudowane (floatmax): \t%.8e\n" floatmax(T)
end


--- Wyznaczanie Największej Liczby Skończonej (MAX) ---
--- Typ: Float16 ---
Iteracyjnie: 		6.55040000e+04
Wbudowane (floatmax): 	6.55040000e+04
--- Typ: Float32 ---
Iteracyjnie: 		3.40282347e+38
Wbudowane (floatmax): 	3.40282347e+38
--- Typ: Float64 ---
Iteracyjnie: 		1.79769313e+308
Wbudowane (floatmax): 	1.79769313e+308


### 3. Wyniki i interpretacja
**Interpretacja i porównanie z `float.h` (C):**
Wyniki iteracyjne są identyczne z wartościami `floatmax(T)`. Porównanie ze stałymi C:

* **Float32 (Single):**
    * Nasz wynik: `3.40282347e+38`
    * Stała C `FLT_MAX`: `3.40282347e+38`
    * *Zgodność: Tak*

* **Float64 (Double):**
    * Nasz wynik: `1.79769313e+308`
    * Stała C `DBL_MAX`: `1.79769313e+308`
    * *Zgodność: Tak*

### 4. Wnioski
Wyznaczona wartość $MAX$ reprezentuje górną granicę zakresu liczb znormalizowanych. Jawne porównanie potwierdza pełną zgodność ze **standardem IEEE 754** oraz stałymi C (`FLT_MAX`, `DBL_MAX`). W **obliczeniach maszynowych**, przekroczenie tej wartości skutkuje "nadmiarem" (overflow) i jest sygnalizowane wartością `Inf`. Zrozumienie tego limitu jest niezbędne przy projektowaniu stabilnych numerycznie algorytmów.
